### Experiment settings

In [ ]:
from private_billing.server import TCPAddress

edge_address = TCPAddress("0.0.0.0", 5555)

In [ ]:
import zmq
from private_billing.messages import Message
from private_billing.server import PickleEncoder

ctxt = zmq.Context()
sock = ctxt.socket(zmq.REQ)


def send(msg: Message, target: TCPAddress):
    with sock.connect(str(target)):
        enc = PickleEncoder.encode(msg)
        sock.send(enc)
        repl = sock.recv()
    return PickleEncoder.decode(repl)

### Seed Exchange
Where these are combined in the `private-billings` library, for the experiment the connect and seed exchange phase are separated, so that one can gather information on the seed exchange phase.
To start the seed exchange phase, execute the following function.

Especially for larger networks, this phase can take some time to finish.
In a network with $ N $ core servers, $ N * (N-1) $  seed messages have to be exchanged.
There are two ways to see whether this phase has concluded
- check the logs and verify that $ N * (N-1) $ `SeedMessage`s are exchanged,
- request telemetry data for cycle round $ -1 $ from the `edge` server. If it has received telemetry from all $ N $ core servers, the exchange phase has concluded.


In [ ]:
from src.experiment.experiment import BootStrapMessage

def bootstrap(edge: TCPAddress):
    msg = BootStrapMessage(None)
    resp = send(msg, edge)
    print(resp)
bootstrap(edge_address)

### Start Billing Cycle
The experiment uses the broadcast of a `ContextMessage(..., CycleContext(cycle_id, ...))` to trigger the billing for cycle `cycle_id`.
After sending this context to the `edge` server, the `edge` broadcasts this to all `core` servers in the network.
Upon receipt, a `core` server then loads its billing data from disk and sends this to the `edge`.
Once the edge has received `Data` from all cores, it commences billing, and eventually sends each `core` their bill.

In [ ]:
from private_billing.messages import ContextMessage
from private_billing.core import CycleContext, vector

def send_context(cycle, edge: TCPAddress):
    cyc_len = 744
    cyc = CycleContext(
        cycle,
        cyc_len,
        vector.new(cyc_len, 0.21),
        vector.new(cyc_len, 0.05),
        vector.new(cyc_len, 0.11),
    )
    msg = ContextMessage(None, cyc)
    send(msg, edge)

### Get telemetry data
To get telemetry data (experiment results), request this data from the `edge` server with the following function.

In [ ]:
from pathlib import Path
from src.experiment.telemetry import GetTelemetryMessage, TelemetryMessage

def get_telemetry(cycle_id):
    msg = GetTelemetryMessage(None, cycle_id)
    resp: TelemetryMessage = send(msg, edge_address)
    entries = resp.value
    
    if cycle_id == -1:
        nr_cores = len(entries)
    else:
        nr_cores = (len(entries) - 1) // 2
        
    fname = Path(f"telemetry-nr_cores_{nr_cores}_round_{cycle_id}.txt")
    
    with fname.open('w') as fp:        
        fp.writelines([str(entry) + "\n" for entry in entries])
    

In [ ]:
# run experiment
import time

for i in range(10):
    send_context(i, edge_address)
    time.sleep(20)
    get_telemetry(i)